In [2]:
import pandas as pd 

In [3]:
# read csv file
bt = pd.read_csv('bt.csv', sep=';')	
sp = pd.read_csv('sp.csv', sep=';')
lu = pd.read_csv('lu.csv', sep=';')

# Data treatment

In [4]:
bt.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,bt-mz.D.8,00:02:25,00:02:24,00:02:25
1,4,12,16,bt-mz.D.12,00:01:52,00:01:52,00:01:52
2,4,16,12,bt-mz.D.16,00:01:51,00:01:50,00:01:50
3,4,24,8,bt-mz.D.24,00:01:39,00:01:39,00:01:39
4,4,32,6,bt-mz.D.32,00:01:46,00:01:45,00:01:45


In [5]:
sp.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,sp-mz.D.8,00:01:29,00:01:30,00:01:30
1,4,12,16,sp-mz.D.12,00:01:22,00:01:20,00:01:22
2,4,16,12,sp-mz.D.16,00:01:06,00:01:07,00:01:08
3,4,24,8,sp-mz.D.24,00:00:59,00:00:57,00:00:57
4,4,32,6,sp-mz.D.32,00:00:58,00:00:58,00:00:59


In [6]:
lu.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,lu-mz.D.8,00:02:31,00:02:30,00:02:32
1,4,12,16,lu-mz.D.12,00:23:33,00:23:40,00:23:25
2,4,16,12,lu-mz.D.16,00:02:36,00:02:37,00:02:38
3,6,12,24,lu-mz.D.12,00:18:19,00:18:27,00:18:37
4,8,16,24,lu-mz.D.16,00:01:17,00:01:16,00:01:16


In [7]:
# change 'partition' column to string
bt['partition'] = bt['partition'].astype(str)
sp['partition'] = sp['partition'].astype(str)
lu['partition'] = lu['partition'].astype(str)

In [8]:
# change elapsed1, elapsed2, elapsed3 to float in seconds
for n in range(0,3):
    bt[f'elapsed{n+1}'] = bt[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
    sp[f'elapsed{n+1}'] = sp[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))
    lu[f'elapsed{n+1}'] = lu[f'elapsed{n+1}'].apply(lambda x: sum(int(i) * 60 ** (2-j) for j, i in enumerate(x.split(':'))))


In [9]:
bt.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,bt-mz.D.8,145,144,145
1,4,12,16,bt-mz.D.12,112,112,112
2,4,16,12,bt-mz.D.16,111,110,110
3,4,24,8,bt-mz.D.24,99,99,99
4,4,32,6,bt-mz.D.32,106,105,105


In [10]:
sp.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,sp-mz.D.8,89,90,90
1,4,12,16,sp-mz.D.12,82,80,82
2,4,16,12,sp-mz.D.16,66,67,68
3,4,24,8,sp-mz.D.24,59,57,57
4,4,32,6,sp-mz.D.32,58,58,59


In [11]:
lu.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3
0,4,8,24,lu-mz.D.8,151,150,152
1,4,12,16,lu-mz.D.12,1413,1420,1405
2,4,16,12,lu-mz.D.16,156,157,158
3,6,12,24,lu-mz.D.12,1099,1107,1117
4,8,16,24,lu-mz.D.16,77,76,76


# Mean and standart deviation elapsed time

In [12]:
# add a new column to the bt dataframe with the name 'mean elapsed t' as the result of the mean of the 'elapsed1', 'elapsed2' and 'elapsed3' columns
bt['mean elapsed t'] = bt[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
sp['mean elapsed t'] = sp[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
lu['mean elapsed t'] = lu[['elapsed1', 'elapsed2', 'elapsed3']].mean(axis=1)
# add a new column to the bt dataframe with the name 'sd elapsed t' as the result of the standart deviation of the 'elapsed1', 'elapsed2' and 'elapsed3' columns
bt['sd elapsed t'] = bt[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)
sp['sd elapsed t'] = sp[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)
lu['sd elapsed t'] = lu[['elapsed1', 'elapsed2', 'elapsed3']].std(axis=1)

In [13]:
bt.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,bt-mz.D.8,145,144,145,144.666667,0.57735
1,4,12,16,bt-mz.D.12,112,112,112,112.000000,0.00000
2,4,16,12,bt-mz.D.16,111,110,110,110.333333,0.57735
3,4,24,8,bt-mz.D.24,99,99,99,99.000000,0.00000
4,4,32,6,bt-mz.D.32,106,105,105,105.333333,0.57735


In [14]:
sp.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,sp-mz.D.8,89,90,90,89.666667,0.577350
1,4,12,16,sp-mz.D.12,82,80,82,81.333333,1.154701
2,4,16,12,sp-mz.D.16,66,67,68,67.000000,1.000000
3,4,24,8,sp-mz.D.24,59,57,57,57.666667,1.154701
4,4,32,6,sp-mz.D.32,58,58,59,58.333333,0.577350


In [15]:
lu.head()

,nodes,ntasks,ntasks-per-node,partition,elapsed1,elapsed2,elapsed3,mean elapsed t,sd elapsed t
0,4,8,24,lu-mz.D.8,151,150,152,151.000000,1.000000
1,4,12,16,lu-mz.D.12,1413,1420,1405,1412.666667,7.505553
2,4,16,12,lu-mz.D.16,156,157,158,157.000000,1.000000
3,6,12,24,lu-mz.D.12,1099,1107,1117,1107.666667,9.018500
4,8,16,24,lu-mz.D.16,77,76,76,76.333333,0.577350


# Theoretic speedup

In [118]:
def get_speed_up(mode):
    it = []; speed_up = []
    for node in [4,6,8]:
        aux = 3
        header = True
        file = f'./Out/{mode}-{node}-1.out'
        with open(file, 'r') as f:
            found = False
            for exe in range(8) if mode == 'bt' or mode == 'sp' else range(aux):
                # ignore the first 6 lines
                for i in range(6) if header is True else range(3):
                    f.readline()
                header = False
                # read the next line
                line = f.readline()
                # get the number after the word 'Iterations:'
                if not found:
                    l = 2 if mode == 'bt' or mode == 'lu' else 7
                    it_exe = int(line.split(' ')[l])
                    found = True
                it.append(it_exe)
                # ignore the next 5 lines
                for i in range(5):
                    f.readline()
                # read the next line
                line = f.readline()
                # get the last number in the line
                speed_up_exe = float(line.split(' ')[-1])
                speed_up.append(speed_up_exe)
                # ignore the next 75 lines
                for i in range(63):
                    f.readline()
                if mode == 'sp':
                    for i in range(13):
                        f.readline()
                elif mode == 'lu':
                    for i in range(6):
                        f.readline()
        print(f'end mode {mode} with {node} nodes')
        aux = 1
    return it, speed_up

In [119]:
# get the speed up for each mode
# it_bt, speed_up_bt = get_speed_up('bt')
# it_sp, speed_up_sp = get_speed_up('sp')
it_lu, speed_up_lu = get_speed_up('lu')

end mode lu with 4 nodes


ValueError: could not convert string to float: ''

# Seedup and efficiency